In [ ]:
# Check GPU
import torch
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))
else:
    print('Enable GPU in Runtime > Change runtime type')


In [ ]:
# Option A: Mount Google Drive and use existing project in Drive (edit the path)
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

PROJECT_DIR = '/content/drive/MyDrive/your_project_path/genai_A1'  # <-- EDIT THIS
Q1_DIR = os.path.join(PROJECT_DIR, 'Q1')

%cd $Q1_DIR
os.environ['PYTHONPATH'] = Q1_DIR
!pwd && ls -la


In [ ]:
# Option B: Clone from GitHub (replace with your repo URL)
import os

%cd /content
!git clone https://github.com/your/repo.git genai_A1
%cd /content/genai_A1/Q1
os.environ['PYTHONPATH'] = '/content/genai_A1/Q1'
!pwd && ls -la


In [ ]:
# Option C: Upload a ZIP and unzip to /content
from google.colab import files
import os

uploaded = files.upload()  # upload your project zip
zip_name = list(uploaded.keys())[0]
!unzip -o "$zip_name" -d /content
%cd /content/genai_A1/Q1
os.environ['PYTHONPATH'] = '/content/genai_A1/Q1'
!pwd && ls -la


In [ ]:
# Install dependencies
%pip install -U pip
%pip install -r /content/genai_A1/requirements.txt

# Optional: Speed up HF datasets cache
import os
os.environ['HF_DATASETS_CACHE'] = '/content/hf_cache'
!mkdir -p /content/hf_cache


In [ ]:
# Ensure we are in Q1 dir and outputs exists
import os, pathlib
Q1_DIR = os.getcwd()
print('Working dir:', Q1_DIR)
os.makedirs('outputs', exist_ok=True)
!ls -la


In [ ]:
# 1) Run Ablation Study (adjust epochs for speed/quality)
!python -m src.ablation_study --outdir outputs --epochs 10

# Show ablation outputs
!ls -la outputs | sed -n '1,120p'


In [ ]:
# 2) Train with Optimal Hyperparameters and compare with Baseline
!python -m src.train_optimal --outdir outputs --epochs 20 --best_params_file outputs/best_hyperparameters.json

# Preview comparison results
import pandas as pd
cmp = pd.read_csv('outputs/model_comparison.csv')
cmp


In [ ]:
# 3) Evaluate Optimal Model on Test Set
!python -m src.evaluate --model_path outputs/optimal_model.pt --num_layers 3 --base_filters 32 --batch_size 64 --outdir outputs

# Show metrics
import pandas as pd
metrics = pd.read_csv('outputs/metrics.csv')
per_class = pd.read_csv('outputs/per_class_metrics.csv')
metrics, per_class.head()


In [ ]:
# 4) Visualize Feature Maps (uses outputs/best_model.pt by default)
!python -m src.visualize_features --model_path outputs/optimal_model.pt --num_layers 3 --base_filters 32 --outdir outputs

from IPython.display import Image, display
print('Feature Maps:')
display(Image('outputs/feature_maps.png'))
print('Layer Activations:')
display(Image('outputs/layer_activations.png'))


In [ ]:
# 5) Zip and Download all outputs
!zip -r /content/Q1_outputs.zip outputs
from google.colab import files
files.download('/content/Q1_outputs.zip')
